In [1]:
import pandas as pd
import numpy as np

In [2]:
# constants
N_STORES = 20
N_WEEKS = 156

# set the column names
columns = ("store_num", "year", "week", "product1_sales", "product2_sales", "product1_price", "product2_price", "product1_promo", "product2_promo", "country")

# create variables that store number of rows and columns
n_rows = N_STORES * N_WEEKS
n_cols = len(columns)

# create an empty dataframe to hold the data using np.empty
sales_df = pd.DataFrame(np.empty(shape=(n_rows, n_cols)), columns=columns)

# check 
print(f"Total rows: {sales_df.shape[0]}\nTotal columns: {sales_df.shape[1]}")
sales_df.head()

Total rows: 3120
Total columns: 10


,store_num,year,week,product1_sales,product2_sales,product1_price,product2_price,product1_promo,product2_promo,country
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# create store numbers
store_numbers = range(101, 101 + N_STORES)

countries = ['USA', 'USA', 'USA', 'DEU', 'DEU', 'DEU',
               'DEU', 'DEU', 'GBR', 'GBR', 'GBR', 'BRA',
               'BRA', 'JPN', 'JPN', 'JPN', 'JPN', 'AUS',
               'CHN', 'CHN']

# map each store a country
store_country = dict(zip(store_numbers, countries))
print(store_country)           

{101: 'USA', 102: 'USA', 103: 'USA', 104: 'DEU', 105: 'DEU', 106: 'DEU', 107: 'DEU', 108: 'DEU', 109: 'GBR', 110: 'GBR', 111: 'GBR', 112: 'BRA', 113: 'BRA', 114: 'JPN', 115: 'JPN', 116: 'JPN', 117: 'JPN', 118: 'AUS', 119: 'CHN', 120: 'CHN'}


In [4]:
# process store_num, year, week and country column
i = 0
for store_num in store_numbers:
    for year in [1, 2, 3]:
        for week in range(1, 53):
            sales_df.loc[i, "store_num"] = store_num
            sales_df.loc[i, "year"] = year
            sales_df.loc[i, "week"] = week
            sales_df.loc[i, "country"] = store_country[store_num]
            i += 1

In [7]:
sales_df.shape

(3120, 10)

In [8]:
# set seed number to generate replicable random data
np.random.seed(143)

# 10% promoted
sales_df["product1_promo"] = np.random.binomial(n=1, p=0.1, size=n_rows)

# 15% promoted
sales_df["product2_promo"] = np.random.binomial(n=1, p=0.15, size=n_rows)

In [9]:
sales_df["product2_promo"].value_counts()

0    2661
1     459
Name: product2_promo, dtype: int64

In [10]:
# each product is sold at one of seven distinct price points ranging from $19.25 to $22.50
price1 = [19.25, 19.75, 20.25, 20.75, 21.25, 21.75, 22.25]
price2 = [19.50, 20, 20.50, 21, 21.50, 22, 22.50]

# use sample with replacement to draw a price for each week
sales_df["product1_price"] = np.random.choice(price1, size=n_rows, replace=True)
sales_df["product2_price"] = np.random.choice(price2, size=n_rows, replace=True)

In [11]:
sales_df.sample(5)

,store_num,year,week,product1_sales,product2_sales,product1_price,product2_price,product1_promo,product2_promo,country
1090,107.0,3.0,51.0,0.0,0.0,19.75,20.5,0,0,DEU
1961,113.0,2.0,38.0,0.0,0.0,20.25,20.5,0,0,BRA
1677,111.0,3.0,14.0,0.0,0.0,19.25,20.5,0,0,GBR
975,107.0,1.0,40.0,0.0,0.0,21.25,21.0,0,0,DEU
3092,120.0,3.0,25.0,0.0,0.0,20.25,20.5,0,0,CHN


In [12]:
# use poisson distribution to simulate sales in units, set product 1 lambda higher than product 2
sales_product1 = np.random.poisson(lam=120, size=n_rows)
sales_product2 = np.random.poisson(lam=100, size=n_rows)

In [13]:
# scale the sales units up or down according to the relative prices.
# price effects often follow a logarithmic function rather than a linear function, so use np.log(price)
log_product1_price = np.log(sales_df["product1_price"])
log_product2_price = np.log(sales_df["product2_price"])

In [14]:
# assume that sales vary as the inverse ratio of prices
# sales of Product 1 go up to the degree that the log(price) of Product 1 is lower than the log(price) of Product 2
sales_product1 = sales_product1 * log_product2_price/log_product1_price
sales_product2 = sales_product2 * log_product1_price/log_product2_price

In [15]:
# sales get a 30 or 40% lift when each product is promoted in store
# the floor() function to drop fractional values & ensure integer counts for weekly unit sales
sales_df["product1_sales"] = np.floor(sales_product1 * (1 + (sales_df["product1_promo"] * 0.3)))
sales_df["product2_sales"] = np.floor(sales_product2 * (1 + (sales_df["product2_promo"] * 0.4)))

In [16]:
sales_df.sample(5)

,store_num,year,week,product1_sales,product2_sales,product1_price,product2_price,product1_promo,product2_promo,country
459,103.0,3.0,44.0,181.0,90.0,21.25,21.5,1,0,USA
1826,112.0,3.0,7.0,110.0,102.0,20.25,22.0,0,0,BRA
677,105.0,2.0,2.0,154.0,89.0,20.75,19.5,1,0,DEU
1207,108.0,3.0,12.0,121.0,104.0,20.75,20.0,0,0,DEU
2583,117.0,2.0,36.0,144.0,92.0,20.25,21.5,0,0,JPN


In [17]:
sales_df.dtypes

store_num         float64
year              float64
week              float64
product1_sales    float64
product2_sales    float64
product1_price    float64
product2_price    float64
product1_promo      int64
product2_promo      int64
country            object
dtype: object

In [18]:
# convert store_num and country data types to categorical
sales_df["store_num"] = sales_df["store_num"].astype(pd.CategoricalDtype())
sales_df["country"] = sales_df["country"].astype(pd.CategoricalDtype())

In [19]:
sales_df.dtypes

store_num         category
year               float64
week               float64
product1_sales     float64
product2_sales     float64
product1_price     float64
product2_price     float64
product1_promo       int64
product2_promo       int64
country           category
dtype: object

In [20]:
sales_df.sample(5)

,store_num,year,week,product1_sales,product2_sales,product1_price,product2_price,product1_promo,product2_promo,country
1562,111.0,1.0,3.0,121.0,107.0,21.75,20.0,0,0,GBR
1193,108.0,2.0,50.0,118.0,108.0,20.75,21.0,0,0,DEU
2918,119.0,3.0,7.0,116.0,99.0,21.75,21.0,0,0,CHN
1775,112.0,2.0,8.0,111.0,85.0,21.75,20.5,0,0,BRA
476,104.0,1.0,9.0,135.0,97.0,19.25,19.5,0,0,DEU
